In [1]:
ATTRIBUTES = {"race": 1,
              "gender": 2,
              "age": 3,
              "skintone": 4,
              "emotion": 5,
              "masked": 6}
attribute_name = "race"
attrs = ATTRIBUTES[attribute_name]

In [2]:
import albumentations as A
from albumentations.pytorch import ToTensorV2 as ToTensor


def get_img_trans(phase,
                  image_size=256,
                  crop_size=224,
                  mean=(0.485, 0.456, 0.406),
                  std=(0.229, 0.224, 0.225)):
    normalize = A.Normalize(mean=mean, std=std)
    if phase == "train":
        return A.Compose(
            [
                A.Resize(image_size, image_size),
                A.RandomCrop(crop_size, crop_size),
                A.HorizontalFlip(),
                normalize,
                ToTensor(),
            ]
        )
    elif phase in ["test", "val"]:
        return A.Compose(
            [
                A.Resize(image_size, image_size),
                A.CenterCrop(crop_size, crop_size),
                normalize,
                ToTensor(),
            ]
        )
    else:
        raise KeyError


In [33]:
import os.path as osp
import numpy as np
import pandas as pd
import cv2

import torch
from torch.utils.data import Dataset


class FaceDataset(Dataset):
    """Fashion Color dataset."""

    def __init__(self,
                 root_dir,
                 image_list,
                 mean,
                 std,
                 image_size,
                 crop_size,
                 mode="train",
                 transform=None):
        """
        Arguments:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.data = pd.read_csv(image_list, delimiter=" ", header=None)
        self.data = np.array(self.data)
        self.root_dir = root_dir
        if transform:
            self.transform = get_img_trans(mode,
                                           image_size=image_size,
                                           crop_size=crop_size,
                                           mean=mean,
                                           std=std)
        self.age_classes = 6
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img_path = osp.join(self.root_dir, self.data[idx][0])
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = self.data[idx][1:]
        target = {"race": label[0],
                  "gender": label[1],
                  "age": label[2],
                  "skintone": label[3],
                  "emotion": label[4],
                  "masked": label[5]}
        if self.transform:
            image = self.transform(image=image)["image"]
        label = self._transform_ages_to_one_hot_ordinal(target,
                                                        self.age_classes
                                                        )
        return image, label

    def _transform_ages_to_one_hot_ordinal(self, target, age_classes):
        age = target["age"]
        new_age = np.zeros(shape=age_classes)
        new_age[:age] = 1
        target["age"] = new_age
        return target
        

In [34]:
from typing import Any, Dict, Optional, Tuple

import torch
from lightning import LightningDataModule
from torch.utils.data import DataLoader, Dataset, random_split


class FashionDataModule(LightningDataModule):
    """`LightningDataModule` for the Fashion-Color dataset."""

    def __init__(
        self,
        root_dir: str = "data/interim",
        image_train_list: str = "face.csv",
        image_test_list: str = "face.csv",
        val_test_split: Tuple[float, float] = (0.5, 0.5),
        batch_size: int = 64,
        num_workers: int = 0,
        pin_memory: bool = False,
        mean: Tuple[float, float, float] = (0.485, 0.456, 0.406),
        std: Tuple[float, float, float] = (0.229, 0.224, 0.225),
        image_size: int = 256,
        crop_size: int = 224
    ) -> None:
        """Initialize a `FashionDataModule`.

        :param data_dir: The data directory. Defaults to `"data/"`.
        :param train_val_test_split: The train, validation and test split. Defaults to `(55_000, 5_000, 10_000)`.
        :param batch_size: The batch size. Defaults to `64`.
        :param num_workers: The number of workers. Defaults to `0`.
        :param pin_memory: Whether to pin memory. Defaults to `False`.
        """
        super().__init__()

        # this line allows to access init params with 'self.hparams' attribute
        # also ensures init params will be stored in ckpt
        self.save_hyperparameters(logger=False)

        self.data_train: Optional[Dataset] = None
        self.data_val: Optional[Dataset] = None
        self.data_test: Optional[Dataset] = None

        self.batch_size_per_device = batch_size

    @property
    def num_classes(self) -> int:
        """Get the number of classes.

        :return: The number of different colors (15).
        """
        return 15

    def prepare_data(self) -> None:
        """Download data if needed. Lightning ensures that `self.prepare_data()` is called only
        within a single process on CPU, so you can safely add your downloading logic within. In
        case of multi-node training, the execution of this hook depends upon
        `self.prepare_data_per_node()`.

        Do not use it to assign state (self.x = y).
        """
        pass

    def setup(self, stage: Optional[str] = None) -> None:
        """Load data. Set variables: `self.data_train`, `self.data_val`, `self.data_test`.

        This method is called by Lightning before `trainer.fit()`, `trainer.validate()`, `trainer.test()`, and
        `trainer.predict()`, so be careful not to execute things like random split twice! Also, it is called after
        `self.prepare_data()` and there is a barrier in between which ensures that all the processes proceed to
        `self.setup()` once the data is prepared and available for use.

        :param stage: The stage to setup. Either `"fit"`, `"validate"`, `"test"`, or `"predict"`. Defaults to ``None``.
        """
        # Divide batch size by the number of devices.
        if self.trainer is not None:
            if self.hparams.batch_size % self.trainer.world_size != 0:
                raise RuntimeError(
                    f"Batch size ({self.hparams.batch_size}) is not divisible by the number of devices ({self.trainer.world_size})."
                )
            self.batch_size_per_device = (
                self.hparams.batch_size // self.trainer.world_size
            )

        # load and split datasets only if not loaded already
        if not self.data_train and not self.data_val and not self.data_test:
            self.data_train = FaceDataset(
                self.hparams.root_dir,
                self.hparams.image_train_list,
                self.hparams.mean,
                self.hparams.std,
                self.hparams.image_size,
                self.hparams.crop_size,
                mode="train",
                transform=True,
            )
            testset = FaceDataset(
                self.hparams.root_dir,
                self.hparams.image_test_list,
                self.hparams.mean,
                self.hparams.std,
                self.hparams.image_size,
                self.hparams.crop_size,
                mode="test",
                transform=True,
            )

            val_size = int(self.hparams.val_test_split[0] * len(testset))
            test_size = len(testset) - val_size

            self.data_val, self.data_test = random_split(
                dataset=testset,
                lengths=[val_size, test_size],
                generator=torch.Generator().manual_seed(42),
            )

    def train_dataloader(self) -> DataLoader[Any]:
        """Create and return the train dataloader.

        :return: The train dataloader.
        """
        return DataLoader(
            dataset=self.data_train,
            batch_size=self.batch_size_per_device,
            num_workers=self.hparams.num_workers,
            pin_memory=self.hparams.pin_memory,
            shuffle=True,
        )

    def val_dataloader(self) -> DataLoader[Any]:
        """Create and return the validation dataloader.

        :return: The validation dataloader.
        """
        return DataLoader(
            dataset=self.data_val,
            batch_size=self.batch_size_per_device,
            num_workers=self.hparams.num_workers,
            pin_memory=self.hparams.pin_memory,
            shuffle=False,
        )

    def test_dataloader(self) -> DataLoader[Any]:
        """Create and return the test dataloader.

        :return: The test dataloader.
        """
        return DataLoader(
            dataset=self.data_test,
            batch_size=self.batch_size_per_device,
            num_workers=self.hparams.num_workers,
            pin_memory=self.hparams.pin_memory,
            shuffle=False,
        )

    def teardown(self, stage: Optional[str] = None) -> None:
        """Lightning hook for cleaning up after `trainer.fit()`, `trainer.validate()`,
        `trainer.test()`, and `trainer.predict()`.

        :param stage: The stage being torn down. Either `"fit"`, `"validate"`, `"test"`, or `"predict"`.
            Defaults to ``None``.
        """
        pass

    def state_dict(self) -> Dict[Any, Any]:
        """Called when saving a checkpoint. Implement to generate and save the datamodule state.

        :return: A dictionary containing the datamodule state that you want to save.
        """
        return {}

    def load_state_dict(self, state_dict: Dict[str, Any]) -> None:
        """Called when loading a checkpoint. Implement to reload datamodule state given datamodule
        `state_dict()`.

        :param state_dict: The datamodule state returned by `self.state_dict()`.
        """
        pass



In [35]:
data_train = FaceDataset(
    "/media/gnort/HDD6/Study/face-analysis-lightning/dataset/face/cropped_faces",
    "/media/gnort/HDD6/Study/face-analysis-lightning/dataset/face/image_lists/face_train.txt",
    [0.485, 0.456, 0.406],
    [0.229, 0.224, 0.225],
    256,
    224,
    mode="train",
    transform=True,
)
dataloader_train = DataLoader(
            dataset=data_train,
            batch_size=32,
            num_workers=4,
            pin_memory=False,
            shuffle=True,
        )


torch.Size([32, 3, 224, 224])
{'race': tensor([0, 0, 0, 1, 2, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0,
        0, 0, 1, 0, 1, 1, 0, 0]), 'gender': tensor([0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 1, 0, 0, 0, 0, 0, 0]), 'age': tensor([[1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
  